<h2>Imports and constants</h2>

In [1]:
import pandas as pd
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data_path = 'data/'

<h2>Functions</h2>

In [3]:
def plot_cities(lons, lats):
    plt.figure(figsize=(8,8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    #ax = plt.axes(projection=ccrs.EuroPP())
    ax.stock_img()
    #ax.coastlines()
    plt.scatter(lons, lats,
             color="blue", marker='o',
             transform=ccrs.Geodetic(),
             )
    plt.show()

In [4]:
#plot_cities(new_df['Longitude'], new_df['Latitude'])

<h2>Merging world cities and cost of living index datasets</h2>

In [5]:
# Load world cities
cities = pd.read_csv(data_path + 'worldcitiespop.csv', sep=",")

/home/filip/anaconda2/envs/geo/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Load cost of living index
coli = pd.read_csv(data_path + 'Cost_of_living_index.csv', sep=",")

In [7]:
# Parse original "City" column to "City" and "Country" columns
# TODO the USA countries are in "NY, New York" now.
coli["City"], coli["Country"] = coli["City"].str.lower().str.split(", ", 1).str

In [8]:
# Take only first 2000 most populated cities and drop any duplicates in city names.
new_cities = cities.sort_values("Population", ascending=False)[0:2000].drop_duplicates("City")

In [9]:
# Check how many cities we have now.
new_cities.shape

(1969, 7)

In [10]:
# Merge new cities and cost of living index datasets.
new_df = pd.merge(new_cities, coli, on="City", how="inner")
new_df.shape

(394, 15)

In [11]:
df = new_df[["City", "Country_y", "Population", "Latitude", "Longitude", "Cost of Living Plus Rent Index", "Local Purchasing Power Index"]]
df.rename(columns={"Country_y": "Country"}, inplace=True)

/home/filip/anaconda2/envs/geo/lib/python2.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [14]:
df.head(10)

,City,Country,Population,Latitude,Longitude,Cost of Living Plus Rent Index,Local Purchasing Power Index
0,tokyo,japan,31480498.0,35.685000,139.751389,61.89,113.09
1,shanghai,china,14608512.0,31.045556,121.399722,42.68,72.16
2,karachi,pakistan,11627378.0,24.905600,67.082200,15.50,36.66
3,delhi,india,10928270.0,28.666667,77.216667,19.23,77.75
4,manila,philippines,10443877.0,14.604200,120.982200,27.46,40.04
5,moscow,russia,10381288.0,55.752222,37.615556,37.33,59.51
6,seoul,south korea,10323448.0,37.598500,126.978300,58.79,102.63
7,sao paulo,brazil,10021437.0,-23.473293,-46.665803,31.11,41.16
8,istanbul,turkey,9797536.0,41.018611,28.964722,25.24,50.13
9,lagos,nigeria,8789133.0,6.453056,3.395833,31.86,19.41


In [15]:
df.shape

(394, 7)

<h2>TODO Merge other datasets to df</h2>